In [ ]:
import os
# import re
import sys
import glob
import pickle
# import shelve
import numpy as np
from scipy.fft import fft, fftfreq
# from scipy.signal import find_peaks
# from scipy.optimize import curve_fit
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.transforms as mtransforms
from matplotlib.colors import LogNorm, FuncNorm
from matplotlib.gridspec import GridSpec
from matplotlib.ticker import FixedLocator, NullLocator, FixedFormatter
# from matplotlib.patches import Polygon
import seaborn as sns

fontsize = 7
lw = 0.75

matplotlib.rc('font', **{'family': 'Times New Roman', 'size': fontsize})
matplotlib.rc('axes', **{'linewidth': 0.75, 'labelsize': fontsize})
matplotlib.rc('xtick', **{'labelsize': fontsize})
matplotlib.rc('ytick', **{'labelsize': fontsize})
matplotlib.rc('xtick.major', **{'width': lw, 'size':3})
matplotlib.rc('ytick.major', **{'width': lw, 'size':3})
matplotlib.rc('ytick.minor', **{'width': lw, 'size':1.5})

%matplotlib inline

if '..' not in sys.path:
    sys.path.append('..')
from dlml.data import load_data_areas

In [ ]:
def plot_correlations(R_mean, R_ctrl_mean, edges, ax, sort_freq=1.0,
                      vmin=None, vmax=None, legend_bbox=[0.4, -0.05]):
    edge = np.abs(edges - sort_freq).argmin()
    kdx = np.argsort(R_mean[edge,:])
    R_mean = R_mean[:,kdx]
    R_ctrl_mean = R_ctrl_mean[:,kdx]

    make_symmetric = False
    if vmin is None:
        vmin = min([r.min() for r in R_mean])
        make_symmetric = True
    if vmax is None:
        vmax = max([r.max() for r in R_mean])
        if make_symmetric:
            if vmax > np.abs(vmin):
                vmin = -vmax
            else:
                vmax = -vmin
    print(f'Color bar bounds: ({vmin:.2f},{vmax:.2f}).')
    ticks = np.linspace(vmin, vmax, 7)
    ticklabels = [f'{tick:.2f}' for tick in ticks]

    cmap = plt.get_cmap('bwr')
    x = np.arange(R_mean.shape[-1])
    y = edges[:-1] + np.diff(edges) / 2
    im = ax[0].pcolormesh(x, y, R_mean, vmin=vmin, vmax=vmax, shading='auto', cmap=cmap)
    ax[0].set_xticks(np.linspace(0, x[-1], 3, dtype=np.int32))
    cbar = plt.colorbar(im, fraction=0.1, shrink=1, aspect=20, label='Correlation',
                        orientation='vertical', ax=ax[0], ticks=ticks)
    cbar.ax.set_yticklabels(ticklabels, fontsize=fontsize-1)
    R_abs_mean = np.mean(np.abs(R_mean), axis=1)
    R_ctrl_abs_mean = np.mean(np.abs(R_ctrl_mean), axis=1)
    ax[1].plot(R_abs_mean, y, 'r', lw=1, label='Tr.')
    ax[1].plot(R_ctrl_abs_mean, y, 'g--', lw=1, label='Untr.')
    ax[1].plot(R_abs_mean - R_ctrl_abs_mean, y, 'k', lw=1, label='Diff.')
    ax[1].legend(loc='lower left', bbox_to_anchor=legend_bbox, frameon=False, fontsize=fontsize-1)

    for i in range(2):
        ax[i].set_ylim(edges[[0,-2]])
        ax[i].set_yscale('log')

In [ ]:
def plot_subplots(N, edges, norm_std, F, Xf, exact_momentum, pred_momentum, pred_momentum_ctrl,
                  MAPE, R_mean, R_ctrl_mean, corr_edges, ax):
    target_values = np.unique(exact_momentum)
    
    tab10 = plt.get_cmap('tab10')
    green, magenta = tab10(2), tab10(6)
    cmap = lambda i: (green, magenta)[i%2]

    for i in range(2):
        ax[0].plot(edges[i][1:], N[i], lw=1, color=cmap(i))
        ax[1].plot(F, 20*np.log10(Xf[i]), lw=1, color=cmap(i))

    ax[0].grid(which='major', axis='x', lw=0.5, ls=':', color=[.6,.6,.6])
    ax[0].set_xlim([-4.5, 4.5])
    ax[0].set_xticks(np.r_[-4 : 4.5 : 2])
    ax[0].set_ylabel('PDF')
    ax[0].set_xlabel('Normalized V')
    ticks = np.r_[0 : 0.61 : 0.2]
    ax[0].set_ylim(ticks[[0,-1]])
    ax[0].yaxis.set_major_locator(FixedLocator(ticks))
    ax[0].yaxis.set_major_formatter(FixedFormatter([f'{tick:g}' for tick in ticks]))
    for i in range(2):
        ax[0].text(-4, 0.6-i*0.1*np.diff(ax[0].get_ylim()), f'STD={norm_std[i]:.2f}', color=cmap(i),
                   fontsize=fontsize-1, va='top')

    ax[1].set_ylim([-55, -10])
    ax[1].set_yticks(np.r_[-50 : -9 : 10])
    ax[1].set_xscale('log')
    ax[1].set_xlabel('Frequency [Hz]')
    ax[1].set_ylabel('Power [dB]')
    f_ticks = np.array([0.1, 0.2, 0.5, 1, 2, 5, 10, 20])
    ax[1].set_xlim(f_ticks[[0,-1]] + np.array([0,1]))
    ax[1].xaxis.set_major_locator(FixedLocator(f_ticks))
    ax[1].xaxis.set_minor_locator(NullLocator())
    ax[1].xaxis.set_major_formatter(FixedFormatter([f'{tick:g}' for tick in f_ticks]))
    ax[1].grid(which='major', axis='y', lw=0.5, ls=':', color=[.6,.6,.6])
    for i,v in enumerate(target_values):
        ax[1].text(0.12, -48-i*0.1*np.diff(ax[1].get_ylim()),
                   r'M={:.2f} GW$\cdot$s$^2$'.format(v),
                   color=cmap(i), fontsize=fontsize-1)

    df = pd.DataFrame(data={'Exact': exact_momentum, 'Pred': np.concatenate(pred_momentum)})
    df_ctrl = pd.DataFrame(data={'Exact': exact_momentum, 'Pred': np.concatenate(pred_momentum_ctrl)})
    sns.violinplot(x='Exact', y='Pred', data=df, cut=0, inner='quartile',
                   palette=[cmap(0), cmap(1)], ax=ax[2], linewidth=0.5)
    ax[2].xaxis.set_major_locator(FixedLocator([0, 1]))
    ax[2].xaxis.set_major_formatter(FixedFormatter([f'{tick:.2f}' for tick in target_values]))
    ax[2].yaxis.set_major_locator(FixedLocator(target_values))
    ax[2].yaxis.set_major_formatter(FixedFormatter([f'{tick:.2f}' for tick in target_values]))
    ax[2].text(np.mean(ax[2].get_xlim()), target_values.mean(), f'MAPE={MAPE:.1f}%',
               ha='center', va='center', fontsize=fontsize-1)

    plot_correlations(R_mean, R_ctrl_mean, corr_edges, sort_freq=[1.1], ax=ax[3:])
    for i in (3,4):
        ax[i].set_ylim(f_ticks[[0,-1]] + np.array([0,1]))
        ax[i].yaxis.set_major_locator(FixedLocator(f_ticks))
        ax[i].yaxis.set_minor_locator(NullLocator())
        if i == 3:
            ax[i].yaxis.set_major_formatter(FixedFormatter([f'{tick:g}' for tick in f_ticks]))
        else:
            ax[i].set_yticklabels([])
    ax[3].set_ylabel('Frequency [Hz]')
    ax[3].set_xlabel('Filter #')
    ax[4].set_xlabel('Correlation')

    sns.despine()

In [ ]:
find_best_experiment_IDs = False

if find_best_experiment_IDs:
    from dlml.utils import collect_experiments
    area_measure = 'momentum'
    stoch_load_bus_IDs = []
    rec_bus_IDs = [3]
    H_G1, D, DZA = None, None, None # 500, 2, 0
    best_experiment_IDs = {1: {'Vd_bus3': '474d2016e33b441889ce8b17531487cb',
                               'Vq_bus3': '617188cbcaef4816a8853081fd303ac1'}}
    for area_ID in (1,2):
        if area_ID not in best_experiment_IDs:
            best_experiment_IDs[area_ID] = {}
        for var_name in (f'Vd_bus{rec_bus_IDs[0]}', f'Vq_bus{rec_bus_IDs[0]}'):
            if var_name not in best_experiment_IDs[area_ID]:
                additional_tags = ['ReLU_none', 'converted_from_PowerFactory', 'all_stoch_loads',
                                   var_name.split('_')[0]]
                expts = collect_experiments(area_ID, area_measure=area_measure, D=D, DZA=DZA, \
                                            stoch_load_bus_IDs=stoch_load_bus_IDs, H_G1=H_G1, \
                                            rec_bus_IDs=rec_bus_IDs, additional_tags=additional_tags, \
                                            verbose=False)
                if expts is None or len(expts) == 0:
                    continue
                expt_IDs = list(expts.keys())
                expt_ID = expt_IDs[np.argmin([expt['val_loss'].min() for expt in expts.values()])]
                MAPE = expts[expt_ID]['MAPE']
                loss = expts[expt_ID]['loss']
                val_loss = expts[expt_ID]['val_loss']
                batch_loss = expts[expt_ID]['batch_loss']
                tags = expts[expt_ID]['tags']
                best_experiment_IDs[area_ID][var_name] = expt_ID
                print(f'The best experiment is {expt_ID[:6]} ' + \
                      f'(val_loss = {val_loss.min():.4f}, ' + \
                      f'MAPE = {MAPE:.4f}%).')
else:
    best_experiment_IDs = {1: {'Vd_bus3': '474d2016e33b441889ce8b17531487cb',
                               'Vq_bus3': '617188cbcaef4816a8853081fd303ac1'}, #'4c8bfb605ed74ea2935f3c3ce416b52d'
                           2: {'Vd_bus3': 'fb6e5dd5df00455fb12c97d0daf77d84',
                               'Vq_bus3': 'f19eac813b484b8c8708ebde2cd5b2c0'}}

In [ ]:
rows = 4
fig,ax = plt.subplots(rows, 5, width_ratios=[1, 1.8, 1, 2.2, 1], figsize=(16.5/2.54, 3.75/2.54*rows))

force = False
k = 0
for area_ID in (1,2):
    for var_name in ('Vd_bus3','Vq_bus3'):
        experiment_ID = best_experiment_IDs[area_ID][var_name]
        data_file = f'hist_spectra_acc_corr_area_{area_ID}_{var_name}_{experiment_ID[:6]}.npz'
        if force or not os.path.isfile(data_file):
            set_name = 'training'
            data_dir = '/home/daniele/Research/deep-power/data/IEEE39/converted_from_PowerFactory/' + \
                f'all_stoch_loads/var_H_area_{area_ID}_comp_grid/subset_2'
            data_files = sorted(glob.glob(os.path.join(data_dir, '*' + set_name + '*.h5')))
            generators_areas_map = [['G02','G03','Comp11'],
                             ['G04','G05','G06','G07','Comp21'],
                             ['G08','G09', 'G10','Comp31'],
                             ['G01']]
            generators_Pnom = {'G01': 10000e6, 'G02': 700e6, 'G03': 800e6, 'G04': 800e6, 'G05': 300e6,
                        'G06': 800e6, 'G07': 700e6, 'G08': 700e6, 'G09': 1000e6, 'G10': 1000e6,
                        'Comp11': 100e6, 'Comp21': 100e6, 'Comp31': 100e6}

            ret = load_data_areas({set_name: data_files}, [var_name],
                                  [generators_areas_map[ID-1] for ID in [area_ID]],
                                  generators_Pnom,
                                  area_measure='momentum',
                                  trial_dur=60,
                                  max_block_size=10000,
                                  use_tf=False,
                                  add_omega_ref=True,
                                  use_fft=False)

            t = ret[0]
            X_raw = ret[1][set_name]
            y = ret[2][set_name]
            group_index = [np.where(y == mom)[0] for mom in np.unique(y)]
            n_mom_groups = len(group_index)
            X_mean, X_std = X_raw.mean(axis=(1,2)), X_raw.std(axis=(1,2))
            X = (X_raw - X_mean) / X_std
            X = X.squeeze()
            y = y.squeeze()
            norm_std = [X[idx].std() for idx in group_index]

            dt = np.diff(t[:2])[0]
            N_samples = t.size
            Xf = fft(X)
            Xf = 2.0 / N_samples * np.abs(Xf[:, :N_samples//2])
            F = fftfreq(N_samples, dt)[:N_samples//2]

            Xf = [Xf[idx,:].mean(axis=0) for idx in group_index]
            N,edges = zip(*(np.histogram(X[idx,:], bins=50, density=True) for idx in group_index))

            experiments_path = '../experiments/neural_network/'
            test_results = pickle.load(open(os.path.join(experiments_path, experiment_ID,
                                                         'test_results.pkl'), 'rb'))
            MAPE = test_results['mape_prediction'][0]
            N_bands = 60
            filter_order = 6
            if experiment_ID == '474d2016e33b441889ce8b17531487cb':
                N_trials = 4000
            else:
                N_trials = 1000
            correlations_file = f'correlations_{experiment_ID[:6]}_{N_bands}-bands_64-filters_' + \
                f'36-neurons_{N_trials}-trials_{filter_order}-butter_{var_name}_pool_1_3.npz'
            correlations = np.load(os.path.join(experiments_path, experiment_ID, correlations_file))
            R, R_ctrl = correlations['R'], correlations['R_ctrl']
            R[correlations['p'] > 0.05] = np.nan
            R_ctrl[correlations['p_ctrl'] > 0.05] = np.nan
            R_mean = np.nanmean(R, axis=0)
            R_ctrl_mean = np.nanmean(R_ctrl, axis=0)

            np.savez_compressed(data_file, N=N, edges=edges, F=F, Xf=Xf, norm_std=norm_std,
                                exact_momentum=correlations['exact_momentum'],
                                pred_momentum=correlations['pred_momentum'],
                                pred_momentum_ctrl=correlations['pred_momentum_ctrl'], MAPE=MAPE,
                                R_mean=R_mean, R_ctrl_mean=R_ctrl_mean, corr_edges=correlations['edges'])

        data = np.load(data_file)
        for key in data.files:
            exec(f'{key} = data["{key}"]')

        plot_subplots(N, edges, norm_std, F, Xf, exact_momentum, pred_momentum, pred_momentum_ctrl,
              MAPE, R_mean, R_ctrl_mean, corr_edges, ax[k,:])
        ax[k,2].set_title('Area {}, {}'.format(area_ID, var_name.replace('_',' @ ')), fontsize=fontsize+1)
        xl,yl = ax[k,3].get_xlim(), ax[k,3].get_ylim()
        x,y = xl[0] + np.diff(xl)/20, np.logspace(np.log10(yl[0]), np.log10(yl[1]), 20)[1]
        ax[k,3].text(x, y, experiment_ID[:6], fontsize=fontsize+1, color='k')
        k += 1

only_first = True
if only_first:
    trans = mtransforms.ScaledTranslation(-0.4, -0.05, fig.dpi_scale_trans)
else:
    trans = mtransforms.ScaledTranslation(-0.3, -0.05, fig.dpi_scale_trans)
for i,label in enumerate('ABCD'):
    if only_first:
        ax[i,0].text(0.0, 1.0, label, transform=ax[i,0].transAxes + trans, fontsize=fontsize+2, va='bottom')
    else:
        for j in range(5):
            ax[i,j].text(0.0, 1.0, label+str(j+1), transform=ax[i,j].transAxes + trans,
                         fontsize=fontsize, va='bottom')
fig.tight_layout(pad=0.3)
plt.savefig('hist_spectra_acc_corr_supp.pdf')